# Introduction to Python - Assignment #3 (Pandas)
**Danilo III O. Gonzales** <br> 
MS Green Data Science

## Table of Contents
1. About the Dataset
2. Importing Libraries and CSV
3. Question 1: Which countries have received the highest total funded amounts? How about the lowest total funded amounts?
4. Question 2: Which activities are most common among Kiva borrowers globally?
5. Question 3: Which countries receive the largest average loans? How about countries with smallest average loans?
6. Question 4: In which sectors are female borrowers most represented, and what are the average loan amounts in those sectors? How about male borrowers?
7. Question 5: What are the top sectors with the highest number of borrowers, and what is the average loan amount in each of those sectors?¶
8. Question 6: How long does it typically take for loans to be fully funded after being posted, and which countries have the shortest and longest funding times?
9. References Used

### About the Dataset

<p style="text-align: justify;">
  The <strong>Kiva Loans Dataset</strong> is a publicly available dataset from 
  <a href="https://www.kaggle.com/datasets/kiva/data-science-for-good-kiva-crowdfunding">
  Data Science for Good: Kiva Crowdfunding Challenge on Kaggle</a>. It contains detailed information about microloans facilitated by <a href="https://www.kiva.org/"> Kiva</a>, a nonprofit organization that connects lenders and borrowers worldwide to support small-scale entrepreneurship and poverty alleviation.
</p>

The dataset contains the following key features: <br>
|Column Name|Description|
|---|---|
|id|Unique ID for loan|
|funded_amount|The amount disbursed by Kiva to the field agent(USD)|
|loan_amount|The amount disbursed by the field agent to the borrower(USD)|
|activity|More granular category (e.g., farming, retail, education|
|sector|High level category (e.g., agriculture, services, food)|
|use|Exact usage of loan amount|
|country_code|ISO country code of country in which loan was disbursed|
|country|Full country name of country in which loan was disbursed|
|region|Full region name within the country|
|currency|The currency in which the loan was disbursed|
|partner_id|ID of Kiva's field partner managing the loan|
|posted_time|Date and time when the loan was first posted on the Kiva platform|
|disbursed_time|Date and time when the loan funds were disbursed to the borrower|
|funded_time|Date and time when the loan reached full funding|
|terms_in_months|Duration of the loan (in months)|
|lender_count|Number of lenders who contributed to the loan|
|tags|Keywords associated to the loan transaction|
|borrower_genders|Gender of borrowers|
|repayment_interval|Frequency of repayments (e.g., monthly, irregular)|

### Importing Libraries and CSV

This analysis uses `numpy` for numerical computing (NumPy Developers, n.d.), `pandas` for data manipulation (pandas Development Team, n.d.), and Matplotlib’s `pyplot` for visualization (Matplotlib Developers, n.d.).

In [5]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Load CSV to Data Frame (df)
df = pd.read_csv('kiva_loans.csv')
print(df)

             id  funded_amount  loan_amount             activity  \
0        653051          300.0        300.0  Fruits & Vegetables   
1        653053          575.0        575.0             Rickshaw   
2        653068          150.0        150.0       Transportation   
3        653063          200.0        200.0           Embroidery   
4        653084          400.0        400.0           Milk Sales   
...         ...            ...          ...                  ...   
671200  1340323            0.0         25.0            Livestock   
671201  1340316           25.0         25.0            Livestock   
671202  1340334            0.0         25.0                Games   
671203  1340338            0.0         25.0            Livestock   
671204  1340339            0.0         25.0            Livestock   

                sector                                                use  \
0                 Food            To buy seasonal, fresh fruits to sell.    
1       Transportation  to re

### **Question 1**: Which countries have received the highest total funded amounts? How about the lowest total funded amounts?

In [17]:
# Using groupby () 
grouped_funding_per_country = df.groupby('country')['funded_amount'].sum().sort_values(ascending=False).apply(lambda x: f"{x:,.2f}")

# Display the top 10 highest
print("Top 10 Countries with Highest Total Funded Amounts (USD):")
print(grouped_funding_per_country.head(5))

# Display the bottom 10 lowest
print("\nBottom 10 Countries with Lowest Total Funded Amounts (USD):")
print(grouped_funding_per_country.tail(5))

Top 10 Countries with Highest Total Funded Amounts (USD):
country
Philippines    54,476,375.00
Kenya          32,248,405.00
Peru           30,394,850.00
Paraguay       29,412,700.00
El Salvador    23,357,725.00
Name: funded_amount, dtype: object

Bottom 10 Countries with Lowest Total Funded Amounts (USD):
country
Mauritania        15,000.00
Afghanistan       14,000.00
Vanuatu            9,250.00
Guam                 395.00
Virgin Islands         0.00
Name: funded_amount, dtype: object


### **Question 2**: Which activities are most common among Kiva borrowers globally?

In [21]:
# Count the frequency in terms of activity
activity_count = df['activity'].value_counts().apply(lambda x: f"{x:,.2f}")

# Display the Top 10 Activity
activity_count.head(10)

activity
Farming                      72,955.00
General Store                64,729.00
Personal Housing Expenses    32,448.00
Food Production/Sales        28,106.00
Agriculture                  27,023.00
Pigs                         26,624.00
Retail                       24,771.00
Clothing Sales               22,339.00
Home Appliances              20,267.00
Higher education costs       19,742.00
Name: count, dtype: object

### **Question 3**: Which countries receive the largest average loans?

### **Question 4**: In which sectors are female borrowers most represented, and what are the average loan amounts in those sectors? How about male borrowers?

### **Question 5**: What are the top sectors with the highest number of borrowers, and what is the average loan amount in each of those sectors?

### **Question 6**: How long does it typically take for loans to be fully funded after being posted, and which countries have the shortest and longest funding times?